In [3]:
import datarobot as dr
import datetime
import pandas as pd

In [4]:
#import the training data set

filename = '/data/GDrive/Customers/sn/Datasets/Classification/Dataset_Base.csv'
now = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M')
project_name = 'sn_Classification_{}'.format(now)
proj = dr.Project.create(sourcedata=filename,
                         project_name=project_name)
print('Project ID: {}'.format(proj.id))

Project ID: 614e079827f708c7282800a8


In [ ]:
#set the target feature and run autopilot

proj.set_target(
    "Converted",
    mode=dr.enums.AUTOPILOT_MODE.QUICK,
    metric="LogLoss",
    featurelist_id=proj.get_featurelists()[1].id,
    worker_count=-1
)

proj.wait_for_autopilot()

In [13]:
#get the model that DR recommends for deployment

model = dr.ModelRecommendation.get(
    proj.id,
    dr.enums.RECOMMENDED_MODEL_TYPE.RECOMMENDED_FOR_DEPLOYMENT
).get_model()

model

Model('eXtreme Gradient Boosted Trees Classifier with Early Stopping')

In [14]:
#Deploy the model to DR

#Get the prediction environment for native DR
prediction_server = dr.PredictionServer.list()[0]

#create the deployment

deployment = dr.Deployment.create_from_learning_model(
    model.id, label='sn_Classification', description='Classification Model',
    default_prediction_server_id=prediction_server.id)

#enable drift tracking

deployment.update_drift_tracking_settings(feature_drift_enabled=True)


Deployment(ServiceNow_Classification)

In [17]:
#Score the first drifted file

dr.BatchPredictionJob.score_to_file(
    deployment.id,
    '/data/GDrive/Customers/sn/Datasets/Classification/Dataset_version_V1_MIXED_FEATURES_Null.csv',
    './predicted.csv',
)

BatchPredictionJob(batchPredictions, '614e224c6317867cb03e9f1f', status=INITIALIZING)

In [19]:
import time
time.sleep(7200)

In [ ]:
#README: To make it easier to visualize wait an hour or two before running this set of predictions
#Score the second drifted file

dr.BatchPredictionJob.score_to_file(
    deployment.id,
    '/data/GDrive/Customers/sn/Datasets/Classification/Dataset_versionV2_Dependent_V1_Multiple_Columns_Nulls_Mixed_Numerical_Categorical_With_Outlier.csv',
    './predicted.csv',
)